In [1]:
import numpy as np
import torch

from torch import nn
from torch.nn import functional as F
from PIL import Image
from torchvision import transforms, datasets, models
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from torchsummary import summary

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [3]:
dataset = datasets.CIFAR100(root='data/', train=True, download=True)

def train_valid_split(Xt):
    X_train, X_test = train_test_split(Xt, test_size=0.2, random_state=42)
    return X_train, X_test

class MyOwnCifar(torch.utils.data.Dataset):
   
    def __init__(self, init_dataset, transform=None):
        self._base_dataset = init_dataset
        self.transform = transform

    def __len__(self):
        return len(self._base_dataset)

    def __getitem__(self, idx):
        img = self._base_dataset[idx][0]
        if self.transform is not None:
            img = self.transform(img)
        return img, self._base_dataset[idx][1]

Files already downloaded and verified


In [4]:
trans_actions = transforms.Compose([
                                    transforms.RandomGrayscale(0.2),
                                    transforms.ColorJitter(brightness=.3, hue=.4),
                                    transforms.RandomRotation(degrees=(0,5)),
                                    transforms.ToTensor()])

train_dataset, valid_dataset = train_valid_split(dataset)

train_dataset = MyOwnCifar(train_dataset, trans_actions)
valid_dataset = MyOwnCifar(valid_dataset, transforms.ToTensor())

In [5]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=128,
                          shuffle=True,
                          num_workers=0)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=128,
                          shuffle=False,
                          num_workers=0)

In [6]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.dp_one = nn.Dropout(0.2)
        self.dp_two = nn.Dropout(0.25)
        
        self.bn_one = torch.nn.BatchNorm2d(3)
        self.conv_one = torch.nn.Conv2d(3, 90, 3, padding=1)
        self.bn_two = torch.nn.BatchNorm2d(90)
        self.conv_two = torch.nn.Conv2d(90, 150, 3, padding=1)
        self.bn_three = torch.nn.BatchNorm2d(150)
        self.conv_three = torch.nn.Conv2d(150, 300, 3, padding=1)
        self.bn_four = torch.nn.BatchNorm2d(300)
        self.fc1 = torch.nn.Linear(4800, 824)
        self.fc2 = torch.nn.Linear(824, 224)
        self.out = torch.nn.Linear(224, 100)
        
        
    def forward(self, x):
        x = self.bn_one(x)
        x = self.conv_one(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        x = self.bn_two(x)
        x = self.conv_two(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        x = self.bn_three(x)
        x = self.conv_three(x)
        x = F.leaky_relu(x, 0.1)
        x = F.max_pool2d(x, 2)
        
        x = self.bn_four(x)
        x = x.view(x.size(0), -1)
        x = self.dp_one(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dp_two(x)
        x = self.fc2(x)
        x = torch.sigmoid(x)
        return self.out(x)
       
net = Net()
print(net)

Net(
  (dp_one): Dropout(p=0.2, inplace=False)
  (dp_two): Dropout(p=0.25, inplace=False)
  (bn_one): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_one): Conv2d(3, 90, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn_two): BatchNorm2d(90, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_two): Conv2d(90, 150, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn_three): BatchNorm2d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_three): Conv2d(150, 300, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn_four): BatchNorm2d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=4800, out_features=824, bias=True)
  (fc2): Linear(in_features=824, out_features=224, bias=True)
  (out): Linear(in_features=224, out_features=100, bias=True)
)


In [7]:
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

In [8]:
summary(net.to(device), input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm2d-1            [-1, 3, 32, 32]               6
            Conv2d-2           [-1, 90, 32, 32]           2,520
       BatchNorm2d-3           [-1, 90, 16, 16]             180
            Conv2d-4          [-1, 150, 16, 16]         121,650
       BatchNorm2d-5            [-1, 150, 8, 8]             300
            Conv2d-6            [-1, 300, 8, 8]         405,300
       BatchNorm2d-7            [-1, 300, 4, 4]             600
           Dropout-8                 [-1, 4800]               0
            Linear-9                  [-1, 824]       3,956,024
          Dropout-10                  [-1, 824]               0
           Linear-11                  [-1, 224]         184,800
           Linear-12                  [-1, 100]          22,500
Total params: 4,693,880
Trainable params: 4,693,880
Non-trainable params: 0
---------------------------

In [21]:
num_epochs = 5
net.train()

for epoch in range(num_epochs):  
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        running_right += (labels == torch.max(outputs, 1)[1]).sum()
        
        # выводим статистику о процессе обучения
        if (i != 0) & (i % 100 == 0):    # печатаем каждые 100 mini-batches
            net.eval()
            
            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.3f}. ' \
                  f'Train acc: {running_right / running_items:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0

            test_running_right, test_running_total = 0.0, 0.0
            for i, data in enumerate(valid_loader):
            
                test_outputs = net(data[0].to(device))
                test_running_total += len(data[1])
                test_running_right += (data[1].to(device) == torch.max(test_outputs, 1)[1]).sum()
            
            print(f'Test acc: {test_running_right / test_running_total:.3f}')
        
        net.train()
        
print('Training is finished!')

Epoch [1/5]. Step [101/313]. Loss: 0.033. Train acc: 0.074. Test acc: 0.119
Epoch [1/5]. Step [201/313]. Loss: 0.030. Train acc: 0.126. Test acc: 0.153
Epoch [1/5]. Step [301/313]. Loss: 0.028. Train acc: 0.165. Test acc: 0.185
Epoch [2/5]. Step [101/313]. Loss: 0.025. Train acc: 0.217. Test acc: 0.249
Epoch [2/5]. Step [201/313]. Loss: 0.024. Train acc: 0.237. Test acc: 0.266
Epoch [2/5]. Step [301/313]. Loss: 0.023. Train acc: 0.262. Test acc: 0.288
Epoch [3/5]. Step [101/313]. Loss: 0.022. Train acc: 0.302. Test acc: 0.314
Epoch [3/5]. Step [201/313]. Loss: 0.021. Train acc: 0.320. Test acc: 0.337
Epoch [3/5]. Step [301/313]. Loss: 0.021. Train acc: 0.336. Test acc: 0.351
Epoch [4/5]. Step [101/313]. Loss: 0.019. Train acc: 0.380. Test acc: 0.364
Epoch [4/5]. Step [201/313]. Loss: 0.019. Train acc: 0.382. Test acc: 0.381
Epoch [4/5]. Step [301/313]. Loss: 0.019. Train acc: 0.386. Test acc: 0.385
Epoch [5/5]. Step [101/313]. Loss: 0.017. Train acc: 0.444. Test acc: 0.407
Epoch [5/5].

In [9]:
resnet50 = models.resnet50(pretrained=True)
print(resnet50)

d:\virt_dev_2\venv_2\Scripts\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\virt_dev_2\venv_2\Scripts\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [10]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False


set_parameter_requires_grad(resnet50, True)
resnet50.fc = nn.Linear(2048, 100)
summary(resnet50.to(device), input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [11]:
resnet50 = resnet50.to(device)

In [14]:
train_dataset, valid_dataset = train_valid_split(dataset)

train_dataset = MyOwnCifar(train_dataset, trans_actions)
valid_dataset = MyOwnCifar(valid_dataset, transforms.ToTensor())

In [12]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=128,
                                           shuffle=True,
                                           num_workers=0)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                                           batch_size=128,
                                           shuffle=False,
                                           num_workers=0)

In [13]:
params_to_update = []
for name, param in resnet50.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)


optimizer = torch.optim.Adam(params_to_update, lr=1e-3)
criterion = nn.CrossEntropyLoss()

In [15]:
num_epochs = 5
resnet50.train()

for epoch in range(num_epochs):  
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data[0].to(device), data[1].to(device)

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = resnet50(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        running_right += (labels == torch.max(outputs, 1)[1]).sum()
        
        # выводим статистику о процессе обучения
        if (i != 0) & (i % 100 == 0):    # печатаем каждые 100 mini-batches
            resnet50.eval()
            
            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.3f}. ' \
                  f'Train acc: {running_right / running_items:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0

            test_running_right, test_running_total = 0.0, 0.0
            for i, data in enumerate(valid_loader):
            
                test_outputs = resnet50(data[0].to(device))
                test_running_total += len(data[1])
                test_running_right += (data[1].to(device) == torch.max(test_outputs, 1)[1]).sum()
            
            print(f'Test acc: {test_running_right / test_running_total:.3f}')

        resnet50.train()
        
print('Training is finished!')

Epoch [1/5]. Step [101/313]. Loss: 0.034. Train acc: 0.076. Test acc: 0.142
Epoch [1/5]. Step [201/313]. Loss: 0.031. Train acc: 0.129. Test acc: 0.178
Epoch [1/5]. Step [301/313]. Loss: 0.030. Train acc: 0.147. Test acc: 0.178
Epoch [2/5]. Step [101/313]. Loss: 0.029. Train acc: 0.167. Test acc: 0.187
Epoch [2/5]. Step [201/313]. Loss: 0.029. Train acc: 0.172. Test acc: 0.188
Epoch [2/5]. Step [301/313]. Loss: 0.029. Train acc: 0.170. Test acc: 0.203
Epoch [3/5]. Step [101/313]. Loss: 0.028. Train acc: 0.181. Test acc: 0.201
Epoch [3/5]. Step [201/313]. Loss: 0.029. Train acc: 0.174. Test acc: 0.206
Epoch [3/5]. Step [301/313]. Loss: 0.029. Train acc: 0.179. Test acc: 0.214
Epoch [4/5]. Step [101/313]. Loss: 0.028. Train acc: 0.189. Test acc: 0.206
Epoch [4/5]. Step [201/313]. Loss: 0.028. Train acc: 0.194. Test acc: 0.218
Epoch [4/5]. Step [301/313]. Loss: 0.028. Train acc: 0.194. Test acc: 0.221
Epoch [5/5]. Step [101/313]. Loss: 0.027. Train acc: 0.206. Test acc: 0.223
Epoch [5/5].